#  Машинный перевод с использованием рекуррентных нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-recurrent-neural-network-pytorch/

## Задачи для самостоятельного решения

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install torchmetrics

In [ ]:
import json
import random
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
import numpy as np
import re
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchmetrics as M
from sklearn.metrics import classification_report
from torchtext.data.metrics import bleu_score
from nltk.translate.bleu_score import corpus_bleu

<p class="task" id="1"></p>

### 1
Считайте файлы `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json` (тестовое множество). Для каждого файла создайте по списка: список предложений на русском языке и список предложений на английском языке. Выведите на экран количество примеров в обучающей и тестовой выборке.

- [x] Проверено на семинаре

In [ ]:
with open('/content/drive/MyDrive/data/RuBQ_2.0_train.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

data_train = pd.DataFrame(data)
data_train = data_train[['question_text', 'question_eng']]
data_train.head()

,question_text,question_eng
0,Что может вызвать цунами?,What can cause a tsunami?
1,Кто написал роман «Хижина дяди Тома»?,"Who wrote the novel ""uncle Tom's Cabin""?"
2,Кто автор пьесы «Ромео и Джульетта»?,"Who is the author of the play ""Romeo and Juliet""?"
3,Как называется столица Румынии?,What is the name of the capital of Romania?
4,На каком инструменте играл Джимми Хендрикс?,What instrument did Jimi Hendrix play?


In [ ]:
with open('/content/drive/MyDrive/data/RuBQ_2.0_test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

data_test = pd.DataFrame(data)
data_test = data_test[['question_text', 'question_eng']]
data_test.head()

,question_text,question_eng
0,Какой стране принадлежит знаменитый остров Пасхи?,Which country does the famous Easter island be...
1,С какой музыкальной группой неразрывно связано...,Which music group is Mick Jagger's name inextr...
2,Где находится Летний сад?,Where is the Summer garden?
3,Какой город является столицей Туркмении?,Which city is the capital of Turkmenistan?
4,В каком городе издавалась с 1857 г. А. Герцено...,In which city was the first Russian revolution...


In [ ]:
X_train = [sentence.split() for sentence in data_train['question_text']]
y_train = [sentence.split() for sentence in data_train['question_eng']]

X_test = [sentence.split() for sentence in data_test['question_text']]
y_test = [sentence.split() for sentence in data_test['question_eng']]

print(f"{X_train[0], y_train[0]=} \n")
print(f"{len(X_train)=} {len(y_train)=}")
print(f"{len(X_test)=} {len(y_test)=}")

X_train[0], y_train[0]=(['Что', 'может', 'вызвать', 'цунами?'], ['What', 'can', 'cause', 'a', 'tsunami?']) 

len(X_train)=2330 len(y_train)=2330
len(X_test)=580 len(y_test)=580


<p class="task" id="2"></p>

### 2
Создайте два Vocab на основе загруженных данных: `ru_vocab` для слов на русском языке и `en_vocab` для слов на английском языке (словари создаются на основе обучающего множества). Добавьте в словари специальные токены `<PAD>`, `<SOS>`, `<EOS>`. Выведите на экран количество токенов в полученных словарях. Выведите на экран максимальное кол-во слов в предложениях на русском языке и в предложениях на английском языке (в обучающей выборке).

- [x] Проверено на семинаре

In [ ]:
ru_vocab = build_vocab_from_iterator(X_train, specials=['<UNK>', '<PAD>', '<SOS>', '<EOS>'])
ru_vocab.set_default_index(0)

en_vocab = build_vocab_from_iterator(y_train, specials=['<UNK>', '<PAD>', '<SOS>', '<EOS>'])
en_vocab.set_default_index(0)

print(f"{len(ru_vocab)=}")
print(f"{len(en_vocab)=} \n")
print(f"Максимальная длина RU: {max(len(sentence) for sentence in X_train)}")
print(f"Максимальная длина EN: {max(len(sentence) for sentence in y_train)}")

len(ru_vocab)=6548
len(en_vocab)=5189 

Максимальная длина RU: 25
Максимальная длина EN: 31


<p class="task" id="3"></p>

### 3
Создайте класс `RuEnDataset`. Реализуйте `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины, добавьте в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`. Создайте датасет для обучающей и тестовой выборки.

- [x] Проверено на семинаре

In [ ]:
class RuEnDataset(Dataset):
    def __init__(self, X, y, X_transform, y_transform):
        self.X = X
        self.y = y
        self.X_transform = X_transform
        self.y_transform = y_transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (self.X_transform(self.X[index]), self.y_transform(self.y[index]))

In [ ]:
X_transform = T.Sequential(
    T.VocabTransform(ru_vocab),
    T.Truncate(max_seq_len=10),
    T.AddToken(2, begin=True),
    T.AddToken(3, begin=False),
    T.ToTensor(padding_value=1),
    T.PadTransform(max_length=15, pad_value=1))

y_transform = T.Sequential(
    T.VocabTransform(en_vocab),
    T.Truncate(max_seq_len=10),
    T.AddToken(2, begin=True),
    T.AddToken(3, begin=False),
    T.ToTensor(padding_value=1),
    T.PadTransform(max_length=15, pad_value=1))

In [ ]:
dataset_train = RuEnDataset(X_train, y_train, X_transform, y_transform)
dataset_train[0]

(tensor([   2,   31,  675, 4057, 6433,    3,    1,    1,    1,    1,    1,    1,
            1,    1,    1]),
 tensor([   2,    7,  145,  240,   25, 5064,    3,    1,    1,    1,    1,    1,
            1,    1,    1]))

In [ ]:
dataset_test= RuEnDataset(X_test, y_test, X_transform, y_transform)
dataset_test[0]

(tensor([  2,   8,  22,  46, 135, 112,   0,   3,   1,   1,   1,   1,   1,   1,
           1]),
 tensor([  2,  13,  24,  20,   4,  42,   0, 116,  51,  47,   3,   1,   1,   1,
           1]))

<p class="task" id="4"></p>

### 4
Опишите модель `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Результатом должен являться тензор размера `1 x batch_size x hidden_dim` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на русском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![encoder](https://i0.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.png?w=815&ssl=1)

- [ ] Проверено на семинаре

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, encoder_hidden_size):
      super().__init__()
      self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=1)
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, encoder_hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X)        # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out)     # out: batch x seq x encoder_hidden_size
    return h                 # 1 x batch x encoder_hidden_size

In [ ]:
encoder = Encoder(vocab_size=len(ru_vocab), embedding_dim=128, encoder_hidden_size=512)

encoder_out = encoder(dataset_train[:16][0])
encoder_out.shape # 1 x batch x encoder_hidden_size

torch.Size([1, 16, 512])

<p class="task" id="5"></p>

### 5
Опишите модель `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке). Пропустите через эту модель тензор скрытых состояний кодировщика, полученный в предыдущей задачи, и выведите размер полученного результата на экран. Результатом должен являться тензор размера `batch_size x seq_len x n_en_words` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на английском языке;
* размер выходного слоя равен количеству слов на английском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![decoder](https://i2.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.png?w=899&ssl=1)

- [ ] Проверено на семинаре

In [ ]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, embedding_dim, decoder_hidden_size):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=1)
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, vocab_size)

  def forward(self, encoder_output, labels):
                                     # encoder_output: 1 x batch x encoder_hidden_size
                                             # labels: batch x seq_len
    seq_len = labels.size(1)                         # max_length=15
    input_tokens = labels[:, 0]                      # batch (SOS)
    decoder_hidden = encoder_output[0]               # batch x encoder_hidden_size
    predictions = []

    for _ in range(1, seq_len):
      out = self.emb(input_tokens).relu()            # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x decoder_hidden_size
      out = self.fc(decoder_hidden)                  # batch x n_en_words
      # teacher forcing
      input_tokens = out.argmax(dim=1).detach()      # batch
      predictions.append(out.unsqueeze(1))           # batch x 1 x n_en_words
    predictions = torch.cat(predictions, dim=1)      # batch x seq x n_en_words

    return predictions

In [ ]:
decoder = Decoder(vocab_size=len(en_vocab), embedding_dim=128, decoder_hidden_size=512)
decoder_out = decoder(encoder_out, dataset_train[:16][1])
decoder_out.shape # batch x seq x n_en_words

torch.Size([16, 14, 5189])

<p class="task" id="6"></p>

### 6
Объедините модели `Encoder` и `Decoder` в одну модель `EncoderDecoder`. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Сделайте полученный результат двумерным, объединив две первые размерности: `batch_size * seq_len x n_en_words`. Выведите размерность полученного результата на экран.

- [ ] Проверено на семинаре

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, X, labels):
        encoder_out = self.encoder(X)
        decoder_out = self.decoder(encoder_out, labels)
        return decoder_out

In [ ]:
encoder = Encoder(vocab_size=len(ru_vocab), embedding_dim=128, encoder_hidden_size=512)
decoder = Decoder(vocab_size=len(en_vocab), embedding_dim=128, decoder_hidden_size=512)
model = EncoderDecoder(encoder, decoder)

model_out = model(dataset_train[:16][0], dataset_train[:16][1])
model_out.shape # batch_size x seq_len x n_en_words

torch.Size([16, 14, 5189])

In [ ]:
model_out_2dim = model_out.view(model_out.shape[0] * model_out.shape[1], model_out.shape[2])
model_out_2dim.shape # batch_size * seq_len x n_en_words

torch.Size([224, 5189])

<p class="task" id="7"></p>

### 7
Настройте модель, решив задачу классификации на основе прогнозов модели `EncoderDecoder`. Игнорируйте токен `<PAD>` при расчете ошибки. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
train_loader = DataLoader(dataset_train, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=32, shuffle=True)

In [ ]:
encoder = Encoder(vocab_size=len(ru_vocab), embedding_dim=256, encoder_hidden_size=512)
decoder = Decoder(vocab_size=len(en_vocab), embedding_dim=256, decoder_hidden_size=512)
model = EncoderDecoder(encoder, decoder).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=1)
optimizer = optim.Adam(model.parameters(), lr=0.0005)
epochs = 500

for epoch in tqdm(range(1, epochs + 1)):
    model.train()
    epoch_loss = 0
    total_predictions = 0
    correct_predictions = 0
    for ru, en in train_loader:
        ru = ru.to(device)
        en = en.to(device)
        output = model(ru, en) # batch_size x seq_len x n_en_words
        loss = criterion(output.view(output.shape[0] * output.shape[1], output.shape[2]), en[:, 1:].flatten()) # batch_size * seq_len x n_en_words, batch_size * seq_len
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss += loss.item()

        total_predictions += (en[:, 1:] != 1).sum().item() # без <PAD>
        correct_predictions += ((output.argmax(dim=2) == en[:, 1:]) * (en[:, 1:] != 1)).sum().item() # количество верных предсказываний без <PAD>

    if (epoch % 50) == 0:
        print(f"\nLoss={round(epoch_loss, 5)} Accuracy={round(correct_predictions / total_predictions, 5)}")

        model.eval()
        with torch.no_grad():
            ru = random.choice(dataset_test)[0].to(device)        # seq_len
            predictions = model(ru.unsqueeze(0), ru.unsqueeze(0)) # 1 x seq_len x n_en_words
            en = predictions.argmax(dim=2).squeeze(0)             # seq_len
            print("RU:", ' '.join(ru_vocab.lookup_tokens(ru.tolist()[1:])))
            print("EN:", ' '.join(en_vocab.lookup_tokens(en.tolist())))

 10%|█         | 50/500 [01:26<12:46,  1.70s/it]


Loss=88.35565 Accuracy=0.64538
RU: Какой архитектурный стиль у здания <UNK> музея <UNK> <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>
EN: What is the architectural style of CCTV headquarters of the <EOS> <EOS> <EOS> <EOS>


 20%|██        | 100/500 [02:49<11:13,  1.68s/it]


Loss=20.22086 Accuracy=0.91672
RU: В каком году основан Гугл? <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
EN: In what year was Lermontov founded? <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>


 30%|███       | 150/500 [04:14<11:29,  1.97s/it]

...


Loss=2.73947 Accuracy=0.99203
RU: Какой архитектурный стиль у здания <UNK> музея <UNK> <EOS> <PAD> <PAD> <PAD> <PAD> <PAD>
EN: What is the architectural style of the building of China? <EOS> <EOS> <EOS> <EOS>


 90%|█████████ | 450/500 [12:23<01:18,  1.58s/it]


Loss=2.00633 Accuracy=0.99525
RU: В каком году родилась <UNK> <UNK> <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
EN: In what year was Nadezhda Babkina born? <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>


100%|██████████| 500/500 [13:44<00:00,  1.65s/it]


Loss=1.80828 Accuracy=0.99553
RU: Кто является основателем <UNK> <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
EN: Who is the founder of Sevastopol? <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS> <EOS>


In [ ]:
translations = []
references = []

model.eval()
with torch.no_grad():
    for ru, en in test_loader:
        ru = ru.to(device)
        en = en.to(device)
        predictions = model(ru, en)
        predicted_ids = predictions.argmax(dim=2)
        for i in range(len(predicted_ids)):
            predicted_translation = en_vocab.lookup_tokens(predicted_ids[i].tolist())
            translations.append(predicted_translation[1:])
        reference_ids = en[:, 1:]
        for i in range(len(reference_ids)):
            reference_translation = en_vocab.lookup_tokens(reference_ids[i].tolist())
            references.append([reference_translation])

bleu_score = corpus_bleu(references, translations)
print(f"BLEU Score: {round(bleu_score, 5)}")

BLEU Score: 0.09735


## Обратная связь
- [x] Хочу получить обратную связь по решению